In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
%config IPCompleter.greedy=True

In [ ]:
from matplotlib import pyplot as plt
%pylab inline
import tensorflow as tf

In [ ]:
cause = pd.read_csv(r'E:\Egyetem\tmp\Medical-Relation-Extraction\ground_truth_cause.csv')
treat = pd.read_csv(r'E:\Egyetem\tmp\Medical-Relation-Extraction\ground_truth_treat.csv')

In [ ]:
cause

In [ ]:
cause.describe()

In [ ]:
dk = cause.groupby("expert")
dk.first()

In [ ]:
import math
nan = float('nan')
cause[cause.expert == cause.expert]

In [ ]:
cause[cause.expert == 1]

In [ ]:
treat.head()

In [ ]:
len(cause)

In [ ]:
def extract_labels(df):
    expert = df.expert
    crowd = df.crowd
    label = 0
    if expert == 1:
        label = 1
    elif pd.isnull(expert) and crowd > 0:
        label = 1
    
    return label

In [ ]:
#expert = cause.expert.tolist()
#crowd = cause.crowd.tolist()

#train_label = np.where(expert == 0 or (pd.isnull(expert) and crowd > 0),1,0)

train_label_cause = []
train_label_treat = []

for index, row in cause.iterrows():
    tmp_label = extract_labels(row)
    train_label_cause.append(tmp_label)
    
for index, row in treat.iterrows():
    tmp_label = extract_labels(row)
    train_label_treat.append(tmp_label)
    
len(train_label_treat)

In [ ]:
test_cause_plain = pd.read_excel(r'E:\Egyetem\tmp\Medical-Relation-Extraction\train_dev_test\ground_truth_cause.xlsx')
test_treat_plain = pd.read_excel(r'E:\Egyetem\tmp\Medical-Relation-Extraction\train_dev_test\ground_truth_treat.xlsx')

In [ ]:
test_cause_plain

In [ ]:
test_cause_plain.head()

In [ ]:
train_cause = cause.term1_UMLS + ' ' + cause.UMLS_seed_relation + ' ' + cause.term2_UMLS
train_treat = treat.term1_UMLS + ' ' + treat.UMLS_seed_relation + ' ' + treat.term2_UMLS

In [ ]:
type(test_cause_plain.term1_UMLS[3])

In [ ]:
test_cause_plain = test_cause_plain.astype(str)
test_treat_plain = test_treat_plain.astype(str)

In [ ]:
test_cause = test_cause_plain.term1_UMLS + ' ' + test_cause_plain.UMLS_seed_relation + ' ' + test_cause_plain.term2_UMLS
test_treat = test_treat_plain.term1_UMLS + ' ' + test_treat_plain.UMLS_seed_relation + ' ' + test_treat_plain.term2_UMLS

In [ ]:
test_cause.head()

In [ ]:
train_cause.head()

In [ ]:
test_label_cause = []
test_label_treat = []

for index, row in test_cause_plain.iterrows():
    tmp_label = extract_labels(row)
    test_label_cause.append(tmp_label)
    
for index, row in test_treat_plain.iterrows():
    tmp_label = extract_labels(row)
    test_label_treat.append(tmp_label)
    
len(test_label_cause)

# In the next part, I will use only the *cause* table

## So far I have the following:

- *train_cause* : the sentences I use for training the model
- *train_label_cause* : the matching labels to the training data
- *test_cause* : this contains the testing data
- *test_label_cause* : the matching labels to the testing data

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

lr  = LogisticRegression(n_jobs=-1)

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit(train_cause)
tr_vectors = X.transform(train_cause)
tst_vectors = X.transform(test_cause)

In [ ]:
tr_vectors.shape

In [ ]:
lr.fit(tr_vectors,train_label_cause)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
lr_pred = lr.predict(tst_vectors)
print("Logistic Regression Test accuracy : {}".format(accuracy_score(test_label_cause, lr_pred)))

## Now trying to built a model

In [ ]:
import torch
from sklearn.model_selection import train_test_split as split
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
VOCAB_SIZE = len(X.vocabulary_)
NUM_LABELS = 2

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
tr_data, val_data = split(train_cause, test_size=0.3, random_state=SEED)
tr_labels, val_labels = split(train_label_cause, test_size=0.3, random_state=SEED)

In [ ]:
tr_data_vecs = torch.FloatTensor(X.transform(tr_data).toarray())
val_data_vecs = torch.FloatTensor(X.transform(val_data).toarray())

In [ ]:
tr_data_loader = [(sample, label) for sample, label in zip(tr_data_vecs, tr_labels)]
val_data_loader = [(sample, label) for sample, label in zip(val_data_vecs, val_labels)]

In [ ]:
from torch.utils.data import DataLoader

train_iterator = DataLoader(tr_data_loader,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            )

valid_iterator = DataLoader(val_data_loader,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          )

In [ ]:
from torch import nn

class MRE(nn.Module):
    def __init__(self, n_inputs):
        super(MRE, self).__init__()
    
        self.hidden1 = nn.Linear(n_inputs, 10)
        self.act1 = nn.ReLU()
        
        self.hidden2 = nn.Linear(10, 8)
        self.act2 = nn.ReLU()
        
        self.hidden3 = nn.Linear(8, 1)
        self.act3 = nn.Sigmoid()
    
    def forward(self, bow_vec):
        bow_vec = self.hidden1(bow_vec)
        bow_vec = self.act1(bow_vec)
        
        bow_vec = self.hidden2(bow_vec)
        bow_vec = self.act2(bow_vec)
        
        bow_vec = self.hidden3(bow_vec)
        bow_vec = self.act3(bow_vec)
        return F.log_softmax(bow_vec, dim=1)

In [ ]:
model = MRE(VOCAB_SIZE)

In [ ]:
import torch.optim as optim

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.NLLLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from sklearn.metrics import classification_report
def class_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    rounded_preds = preds.argmax(1)
    correct = (rounded_preds == y).float()
    
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
import torch.nn.functional as F

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for texts, labels in iterator:
        texts = texts.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
                  
        predictions = model(texts)

        loss = criterion(predictions, labels)
        acc = class_accuracy(predictions, labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        
        for texts, labels in iterator:
            
            texts = texts.to(device)
            labels = labels.to(device)
            
            predictions = model(texts)
            loss = criterion(predictions, labels)
            
            acc = class_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
## N_EPOCHS = 50

## best_valid_loss = float('inf')

## for epoch in range(N_EPOCHS):

    ## start_time = time.time()
    
    ## train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    ## valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    ## end_time = time.time()

    ## epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    ## print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    ## print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    ## print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
input_dim = tr_vectors.shape[1]
input_dim

In [ ]:
from keras.callbacks import History 



In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
kerasmodel = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, input_dim=input_dim, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="softmax"),
    tf.keras.layers.Dense(1)
])

kerasmodel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
label_array = np.array(train_label_cause)
label_array_test = np.array(test_label_cause)
history = History()

history = kerasmodel.fit(tr_vectors, label_array, epochs=10, batch_size=64, callbacks=[history],
                         validation_data=(tst_vectors, label_array_test))

In [ ]:
kerasmodel.summary()


In [ ]:
loss, accuracy = kerasmodel.evaluate(tr_vectors, label_array, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
val_loss, val_accuracy = kerasmodel.evaluate(tst_vectors, label_array_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(val_accuracy))

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)